In [114]:
from pydantic import BaseModel, ConfigDict
from typing import List, Dict
import random
import plotly.graph_objects as go
import plotly.colors as pc

In [115]:
class Point(BaseModel):
    coordinates: List[float]

class PythagoreanSupportMachineInput(BaseModel):
    points: List[Point]
    n_groups: int

class Group(BaseModel):
    points: List[Point]
    n_points: int
    centroid: Point

class PythagoreanSupportMachineOutput(BaseModel):
    groups: List[Group]
    n_dimensions: int

class PointStatus(BaseModel):
    probability: float
    distance: float
    point: Point

class Node(BaseModel):
    points_status: List[PointStatus]
    centroid: Point

class MainMatrix(BaseModel):
    nodes: List[Node]

In [116]:
def eject_object(base_model_obj, filename):
    with open(filename, 'w') as f:
        f.write(base_model_obj.model_dump_json())

In [117]:
def generate_points(
    n_dimensions: int,
    n_points: int,
    min_point_range: float,
    max_point_range: float
) -> List[Point]:
    points = []
    for _ in range(n_points):
        coords = [random.uniform(min_point_range, max_point_range) for _ in range(n_dimensions)]
        points.append(Point(coordinates=coords))
    return points


pts = generate_points(n_dimensions=2, n_points=20, min_point_range=-10, max_point_range=10)
input = PythagoreanSupportMachineInput(points=pts, n_groups=10)
eject_object(input, "input.json")

In [118]:
def get_first_points_status(points: List[Point], n_groups: int) -> List[List[PointStatus]]:
    nodes: List[List[PointStatus]] = [[] for _ in range(n_groups)]
    
    for point in points:
        raw_probs = [random.random() for _ in range(n_groups)]
        total = sum(raw_probs)
        
        normalized_probs = [p / total for p in raw_probs]
        
        for i, prob in enumerate(normalized_probs):
            nodes[i].append(PointStatus(probability=prob, point=point.model_dump(), distance=0))
    
    return nodes

def calculate_centroid_coordinate(probabilities: List[PointStatus], dimension: int) -> float:
    coordinate_numerator = 0
    coordinate_denominator = 0
    for probability in probabilities:
        coordinate_numerator += probability.point.coordinates[dimension] * (probability.probability) ** 2
        coordinate_denominator += (probability.probability) ** 2
    return coordinate_numerator / coordinate_denominator

def calculate_centroid(probabilities: List[PointStatus], n_dimensions: int) -> Point:
    coordinates = []
    for i in range(n_dimensions):
        coordinate = calculate_centroid_coordinate(probabilities, i)
        coordinates.append(coordinate)
    return Point(coordinates=coordinates)
    

def set_first_matrix(input: PythagoreanSupportMachineInput) -> MainMatrix:
    n_dimensions = len(input.points[0].coordinates)
    point_status_list = get_first_points_status(input.points, input.n_groups)
    nodes = []
    for points_status in point_status_list:
        centroid = calculate_centroid(points_status, n_dimensions)
        nodes.append(Node(points_status=points_status, centroid=centroid))
    return MainMatrix(nodes=nodes)


fisrt_matrix = set_first_matrix(input)
eject_object(fisrt_matrix, "first_matrix.json")

In [119]:
def calculate_distance(point: Point, centroid: Point) -> float:
    return sum((p - c) ** 2 for p, c in zip(point.coordinates, centroid.coordinates)) ** 0.5

def update_distances_for_node(node: Node) -> MainMatrix:
    for point in node.points_status:
        point.distance = calculate_distance(point.point, node.centroid)
    return node
    
def update_distances(matrix: MainMatrix) -> MainMatrix:
    for node in matrix.nodes:
        update_distances_for_node(node)
    return matrix
    

distances = update_distances(fisrt_matrix)
eject_object(distances, "distances.json")

In [120]:
def calculate_probability(distance: float, points_status: List[PointStatus]) -> float:
    return sum((distance / point.distance)**2 for point in points_status) ** -1

def update_probabilities_for_node(node: Node) -> MainMatrix:
    for point in node.points_status:
        point.probability = calculate_probability(point.distance, node.points_status)
    return node

def update_probabilities(distances: MainMatrix) -> MainMatrix:
    for node in distances.nodes:
        update_probabilities_for_node(node)
    return distances

probabilities = update_probabilities(distances)
eject_object(probabilities, "probabilities.json")

In [121]:
def update_centroids(probabilities: MainMatrix) -> MainMatrix:
    for node in probabilities.nodes:
        node.centroid = calculate_centroid(node.points_status, len(node.points_status[0].point.coordinates))
    return probabilities

centroids = update_centroids(probabilities)
eject_object(centroids, "centroids.json")

In [122]:

def extract_all_points(matrix: MainMatrix) -> List[Point]:
    """Extract the list of distinct points from the first node."""
    return [ps.point for ps in matrix.nodes[0].points_status]


def get_n_dimensions(matrix: MainMatrix) -> int:
    """Return the number of dimensions based on centroid coordinates."""
    return len(matrix.nodes[0].centroid.coordinates)


def find_best_group_for_point(matrix: MainMatrix, point_idx: int) -> int:
    """Determine which node (group) has the highest probability for a given point index."""
    return max(
        range(len(matrix.nodes)),
        key=lambda i: matrix.nodes[i].points_status[point_idx].probability
    )


def assign_points_to_groups(matrix: MainMatrix, points: List[Point]) -> Dict[int, List[Point]]:
    """Assign each point to the node with the highest probability."""
    assignments: Dict[int, List[Point]] = {i: [] for i in range(len(matrix.nodes))}
    for point_idx, point in enumerate(points):
        best_group_idx = find_best_group_for_point(matrix, point_idx)
        assignments[best_group_idx].append(point)
    return assignments


def build_groups(matrix: MainMatrix, assignments: Dict[int, List[Point]]) -> List[Group]:
    """Build Group objects with their assigned points and corresponding centroids."""
    groups: List[Group] = []
    for i, node in enumerate(matrix.nodes):
        group_points = assignments[i]
        group_points = [p.model_dump() for p in group_points]
        centroid = node.centroid.model_dump()
        groups.append(
            Group(
                points=group_points,
                n_points=len(group_points),
                centroid=centroid
            )
        )
    return groups


def produce_output(matrix: MainMatrix) -> PythagoreanSupportMachineOutput:
    """Main transformation entrypoint: build the PythagoreanSupportMachineOutput."""
    points = extract_all_points(matrix)
    n_dimensions = get_n_dimensions(matrix)
    assignments = assign_points_to_groups(matrix, points)
    groups = build_groups(matrix, assignments)

    return PythagoreanSupportMachineOutput(
        groups=groups,
        n_dimensions=n_dimensions
    )


output = produce_output(centroids)
eject_object(output, "output.json")

In [123]:

def plot_groups(output: PythagoreanSupportMachineOutput):
    """
    Plot the grouped points and centroids using Plotly.
    Points and centroids from the same group share the same color.
    Supports any number of groups (colors generated dynamically).
    Raises an exception if dimensions are greater than 2.
    """
    if output.n_dimensions != 2:
        raise ValueError("Can only plot 2-dimensional data.")

    fig = go.Figure()
    n_groups = len(output.groups)

    # Dynamically generate colors from a continuous colormap
    colors = pc.sample_colorscale("Turbo", [i / max(1, n_groups - 1) for i in range(n_groups)])

    for idx, group in enumerate(output.groups):
        color = colors[idx]

        # --- Plot points if any ---
        if group.n_points > 0:
            xs = [p.coordinates[0] for p in group.points]
            ys = [p.coordinates[1] for p in group.points]

            fig.add_trace(go.Scatter(
                x=xs, y=ys,
                mode="markers",
                name=f"Group {idx}",
                marker=dict(size=8, line=dict(width=1), color=color, symbol="circle"),
            ))

        # --- Always plot centroid ---
        cx, cy = group.centroid.coordinates
        fig.add_trace(go.Scatter(
            x=[cx], y=[cy],
            mode="markers+text",
            name=f"Centroid {idx}",
            text=[f"C{idx}"],
            textposition="top center",
            marker=dict(size=14, symbol="x", line=dict(width=2), color=color),
            showlegend=(group.n_points == 0),
        ))

    fig.update_layout(
        title="Pythagorean Support Machine Clusters",
        xaxis_title="X Coordinate",
        yaxis_title="Y Coordinate",
        legend_title="Groups",
        template="plotly_white"
    )

    fig.show()


plot_groups(output)